In [1]:
import os
print(os.getcwd())

/Users/user/Desktop


In [2]:
# os.chdir(r'C:/Users/user/Desktop/Hevrin Data') # r is a raw string, not read as an escape sequence
print(os.getcwd())  # so copied and pasted the required files from IDrive located by a search

/Users/user/Desktop


# I. DATA WRANGLING TO OBTAIN A TIME SERIES WITH A DATETIME INDEX

In [3]:
import pandas as pd
df= pd.read_csv('Hevrin Usage.csv',header= None,skiprows= 17, usecols= [*range(45,70)]) 
# skip rows st start August 1st
# nrows= 730,   nrows st two years to start with: better is all days, then resample to weekly with .resample('W')
print(df.shape)
df.head()

(2788, 25)


,45,46,47,48,49,50,51,52,53,54,...,60,61,62,63,64,65,66,67,68,69
0,08/01/2011,0.77,0.64,0.70,0.58,0.58,0.51,0.70,0.70,0.32,...,1.600,2.18,2.50,2.75,3.58,3.33,3.33,3.33,2.62,1.60
1,08/02/2011,0.77,0.64,0.70,0.70,0.58,0.64,0.70,0.38,0.26,...,2.112,1.66,2.43,3.07,4.29,8.45,6.08,3.71,2.75,1.86
2,08/03/2011,1.22,0.70,0.77,0.70,1.09,0.77,0.83,0.38,0.32,...,1.536,2.43,3.20,3.26,2.69,3.97,3.26,3.46,1.47,0.45
3,08/04/2011,0.38,0.58,0.38,0.38,0.38,0.32,0.45,0.38,0.32,...,0.704,2.05,3.39,3.65,3.33,2.56,1.47,1.09,1.22,0.77
4,08/05/2011,0.51,0.70,0.45,0.38,0.51,0.38,0.51,0.32,0.32,...,1.088,1.86,2.75,3.20,2.50,1.92,1.54,1.22,1.22,0.77


In [4]:
# df.info()

In [5]:
df.columns= ['day','00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']

In [6]:
df= df.set_index(df['day'])
df= df.drop(['day'], axis=1)
df.tail(3)

,00,01,02,03,04,05,06,07,08,09,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
03/17/2019,0.58,0.45,0.58,0.58,0.45,0.51,0.51,0.51,0.96,2.11,...,1.024,0.96,0.96,1.22,2.18,6.78,2.75,2.88,0.83,0.51
03/18/2019,0.45,0.51,0.51,0.51,0.51,0.51,0.58,3.97,3.33,0.70,...,4.352,6.78,2.56,0.96,0.45,0.77,0.90,0.83,2.82,0.58
03/19/2019,0.58,0.51,0.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dft= df.stack()
dft=dft.reset_index()
dft.columns= ['day','hour','kwh'] # where 'day' is string, not DT

In [8]:
dft.head(8)

,day,hour,kwh
0,08/01/2011,00,0.77
1,08/01/2011,01,0.64
2,08/01/2011,02,0.70
3,08/01/2011,03,0.58
4,08/01/2011,04,0.58
5,08/01/2011,05,0.51
6,08/01/2011,06,0.70
7,08/01/2011,07,0.70


In [9]:
dft['date']= pd.to_datetime(dft['day'] + ' ' + dft['hour'] + ':00:00')
from datetime import datetime
dft['weekday']= dft.date.dt.weekday_name
dft['month']= dft.date.dt.month
dft['year']= dft.date.dt.year
dft.set_index(dft['date'], inplace= True)
dft= dft.drop(['day','date'], axis=1)
dft['hour']= dft['hour'].astype('category')
dft['weekday']= dft['weekday'].astype('category')
dft['month']= dft['month'].astype('category')
dft['year']= dft['year'].astype('category')
dft.info()
dft.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 66505 entries, 2011-08-01 00:00:00 to 2019-03-19 02:00:00
Data columns (total 5 columns):
hour       66505 non-null category
kwh        66505 non-null float64
weekday    66505 non-null category
month      66505 non-null category
year       66505 non-null category
dtypes: category(4), float64(1)
memory usage: 1.3 MB


,hour,kwh,weekday,month,year
date,,,,,
2011-08-01 00:00:00,00,0.77,Monday,8,2011
2011-08-01 01:00:00,01,0.64,Monday,8,2011
2011-08-01 02:00:00,02,0.70,Monday,8,2011
2011-08-01 03:00:00,03,0.58,Monday,8,2011
2011-08-01 04:00:00,04,0.58,Monday,8,2011


# Preparation of Seasonal Data

In [10]:
df_17= dft[dft.year== 2017]
df_17.head(3)

,hour,kwh,weekday,month,year
date,,,,,
2017-01-01 00:00:00,00,0.51,Sunday,1,2017
2017-01-01 01:00:00,01,0.51,Sunday,1,2017
2017-01-01 02:00:00,02,0.45,Sunday,1,2017


In [11]:
kwh_17= df_17.kwh.tolist()
W_17= kwh_17[0:2184]
W_17[0:4]
# len(W_17)

[0.51, 0.51, 0.45, 0.51]

In [12]:
# create nested list 13 weeks of 168 hours each for Winter (W)
W17_weekly= [ W_17[i:i+168] for i in range(0,2184,168)] 


In [13]:
# Repeat for 2016 and add to the W Winter list:
df_16= dft[dft.year== 2016]
kwh_16= df_16.kwh.tolist()
W_16= kwh_16[0:2184]
W16_weekly= [W_16[i:i+168] for i in range(0,2184,168)] 

In [14]:
# Repeat for 2015 and add to the W Winter list:
df_15= dft[dft.year== 2015]
kwh_15= df_15.kwh.tolist()
W_15= kwh_15[0:2184]
W15_weekly= [W_15[i:i+168] for i in range(0,2184,168)] 

In [15]:
# Repeat for 2014 and add to the W Winter list:
df_14= dft[dft.year== 2014]
kwh_14= df_14.kwh.tolist()
W_14= kwh_14[0:2184]
W14_weekly= [W_14[i:i+168] for i in range(0,2184,168)] 

In [16]:
W14_weekly[0] 
len(W14_weekly)

13

In [17]:
W_weekly= W17_weekly + W16_weekly + W15_weekly + W14_weekly  # now have 52 Winter weeks to randomly draw 13 from
len(W_weekly)

52

In [18]:
import random
mr1= [random.choice(W_weekly) for i in range(13)]  # m1 is Metering Record 1
mr1[0]
len(mr1) # 13 : so we have a list of lists, drawn at random

13

In [19]:
# We now need to flatten the list-of-lists into a single list of length 168 x 14 weeks
fmr1 = [item for sublist in mr1 for item in sublist]
len(fmr1) # 2184 : a flat list of 13 winter weeks chosen at random 
# We now need to repeat this 100x and add to dataframe

2184

In [20]:
type(fmr1)

list

In [21]:
def Winter_list(list):
    W= []
    for i in range(5):
        a= [random.choice(W_weekly) for p in range(13)]     # List Comprehension to draw 13 weeks of 168 hours
        b= [item for sublist in a for item in sublist] # Flatten each List-of-lists into a single list
        W.append(b)
    return W

In [22]:
c= Winter_list(W_weekly)
len(c)      # 5 since range(5)
len(c[0])   # 2184, since 168 x 13 weeks


2184

In [23]:
import pandas as pd
df= pd.DataFrame(fmr1)  # This should work for a list of lists also!!
df.shape  # (2184,1)
#df.head()  data entered as a single column (but can transpose)


(2184, 1)

In [24]:
# df.append(df2, ignore_index = True) 

In [25]:
# This generates a dataframe from the list of lists; This is exactly what we need!!
df2= pd.DataFrame(c)
df2.shape

(5, 2184)

In [ ]:
# Next add metrics for Classification based on the preceding 2184 hourly observations
df2.average= mean()

In [28]:
df2['average']= df2.sum(axis = 1, skipna = True) 

df2.average[0

# II. EXPLORATORY DATA ANALYSIS

In [ ]:
import matplotlib.pyplot as plt
weekly= dft.kwh.resample('W').sum()
monthly= 0.25 * dft.kwh.resample('M').sum()
plt.plot(weekly)
plt.plot(monthly)
plt.title('Weekly & Monthly Household Power Consumption in kwh')
plt.ylabel('kwh')
plt.show()        #

What is surprising about the above time series is that despite the family's addition of two energy-intensive children (including due to more laundry and cooking), and a small EV in 2018, replaced by a large EV in 2019, the highest weekly peaks occured not in the last year, but rather in 2012 and 2013. 
This could be weather related as hot summers and cold winters can materially increase the load.
Also the hourly peaks are most relevant, not the weekly totals and this matter needs to be investigated quite separately.

In [ ]:
plt.hist(dft.kwh, bins=30, density= True)
plt.title('Histogram of Hourly Power Consumption: August 2012 to Present')
plt.xlabel('Hourly Power Consumption in kwh')
plt.axvline(dft.kwh.mean(), color='k', linestyle='dashed', linewidth=2)
plt.axvline(3.15, color='r', linestyle='dashed', linewidth=3)
plt.show()

In [ ]:
# A Python Dictionary of a selection of Load Factors and the respective number of age-equivalent years is provided here:
# age_acceleration_factors= {0.64:1, 0.70:2.65, 0.75:5.65, 0.80:11.67, 0.85:23.3, 0.90:38.9, 0.95:70.0}

In [ ]:
import seaborn as sns
# plt.figure() explore with shift-tab-tab with cursor positioned within parentheses
plt.figure(figsize= (15,10))    # this now fills the page so detail can be seen!!
sns.boxplot(x= dft.hour,y=dft.kwh)
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.title('Boxplot of Hourly kwh Consumption by the Hour of the Day: August 2012 to Present')
plt.show()

In [ ]:
# seaborn.boxplot(x=None, y=None, hue=None, data=None, order=None, hue_order=None, orient=None, color=None, 
#                palette=None, saturation=0.75, width=0.8, dodge=True, fliersize=5, linewidth=None, whis=1.5, notch=False, ax=None, **kwargs)¶

In [ ]:
plt.figure(figsize= (15,10)) # Have to record this before the boxplot is called, it seems!
sns.boxplot(dft.month, dft.kwh)
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.title('Boxplot of Hourly Power Consumption by Month')

# NB the reason Months 2 & 3 have so many outliers at they include  half a month each of Tesla charging at 10 kwh
# Note also the interquartile ranges are exceeded in the months of June, July,and August: so 6 hours/day > 3.15 kwh
# March, April, May have the lowest loads die to the mild Spring temperatures (same for October in the Fall)

In [ ]:
plt.figure(figsize= (15,10)) 
sns.boxplot(dft.year, dft.kwh)
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.title('Boxplot of Hourly Power Consumption by Year')
# The median consumption in the years 2012 & 2013 is NOT higher than 2014, so perhaps the high weekly 
# consumption level peaks in those years (see above) was due to weather conditions (eg a hot summer and a high A/C load).

# But in fact the years 2018 & 2019 are different
In these two years the upper quartile exceeds the 3.15 threshold, so it appears that a quarter of the time the threshold level for accelerating the ageing of the transformer is exceeded. However, this is only on an AVERAGE basis, and so is perhaps unduly conservative as the other [four] households served may have lower power consumption in that hour. Also in 2019 they have not yet acquired EV's.

In [ ]:
df_12= dft[dft.year== 2012]
df_12.head(3)

In [ ]:
plt.figure(figsize= (15,10)) 
# plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.plot(df_12.kwh)
plt.show()

In [ ]:
plt.figure(figsize= (15,10)) 
plt.axvline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.hist(df_12.kwh, bins=20, density= False) # This shows frequencies so we can surmise > 100 hours exceeded threshold

In [ ]:
plt.figure(figsize= (15,10)) 
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
sns.boxplot(x= df_12.hour,y=df_12.kwh)
# Importantly this shows that from 1pm through 9pm (so nine hours) the interquartile range exceeds the 3.15 threshold
# So if household consumption is correlated (eg due to common very cold and hot temperatures), then premature ageing
# is possible (although these loads are linear) 

In [ ]:
monthly= df_12.kwh.resample('W').sum()

In [ ]:
plt.figure(figsize= (15,10)) 
plt.title('Histogram of Aggregate Monthly Power Consumption: 2012')
plt.ylabel('Power Consumption in kwh')
monthly.plot()

# A Small EV and a Level 1 "Trickle Charger" were Acquired in July, 2018¶
In the chart below, after July we see relatively little "white space" below the red horizontal line, suggesting that the trickle charger was active throughout much of the night.
We used to see daly minimums (during the night) of below 1 kwh, but after July the minimums commonly exceeded 1.75 kwh

In [ ]:
df_18= dft[dft.year== 2018]
plt.figure(figsize= (15,10)) 
df_18.kwh.plot()
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.axhline(5.5, color='g', linestyle='dashed', linewidth=3)

plt.title("Hourly kwh Consumption in 2018: Level 1 Charger acquired in July")
plt.ylabel("kwh")
# plt.axvline(2018-07-01 00:00:00, color='y', linestyle='dashed', linewidth=5)# problem w datetime specification
# # plt.axvline(2018-07-01 , color='g', linestyle='dashed', linewidth=3)
plt.show()

In [ ]:
lows=  (df_18.kwh < 1.75)
sum(lows) # Hmmm this is > 75% of the hours so the chart is quite misleading being dominated by the ink for spikes.

In [ ]:
df_18.info() # hour and month are each strings (and categorical variables)
df_18.tail(3)

In [ ]:
df_18['kwhMA8C']= df_18.kwh.rolling(8,center= True).mean()

In [ ]:
plt.figure(figsize= (15,10))
plt.grid(True)
# plt.plot(df_18.kwh, label= 'kwh')
plt.plot(df_18.kwhMA8C, label= 'Centered 8-hour Moving Average')

In [ ]:
sum(df_18.kwh > 3.15) # so for the whole year 2018, of 8,760 hours just 466 exceeded the hourly threshold of 3.15 kwh

In [ ]:
# df_18.month.isin() # weird 'year' and month behave here like non-strings!!

In [ ]:
pre_L1= df_18[df_18.month.isin([1,2,3,4,5,6])]
post_L1= df_18[df_18.month.isin([7,8,9,10,11,12])]
pre_L1.head()

# One possible Signal for Classifying the new use of a Level 1 Charger is the number of times a certain hourly kwh threshold is exceeded
For example, in the first half of 2018 not once was an hourly consumption of more than 5.5 kwh recorded.
However, after the Level 1 charger was acquired in July, this threshold was exceeded 13 times.

In [ ]:
print(sum(pre_L1.kwh > 5.5))

In [ ]:
print(sum(post_L1.kwh > 5.5))

In [ ]:
# sum(df_18.kwh > 5.5) # this includes just one month and several days of EV ownership
# NEW CODE FOR ROLLING SIGNAL DAILY OR WEEKLY
peak_L1= df_18.kwh > 5.5
signal_L1= peak_L1.resample('M').sum() # Monthly is a more stable indicator: consistently more than 7x per week!
plt.figure(figsize= (15,10))
plt.title('Number of Hours per Week that 5.5 kwh L1 Signal Threshold is Exceeded')
plt.ylabel('Number of Hours')
# plt.axhline(7, color='r', linestyle='dashed', linewidth=2)
plt.plot(signal_L1)
# Exceeded regularly in Summer only due to AC load!

In [ ]:
peak_7year= dft.kwh > 5.5 # all years, rather than a single year
signal_7year= peak_7year.resample('M').sum() # Monthly is a more stable indicator: consistently more than 7x per week!
plt.figure(figsize= (15,10))
plt.title('Number of Hours per Month that 5.5kwh Signal Threshold is Exceeded')
plt.ylabel('Number of Hours')
# plt.axhline(7, color='r', linestyle='dashed', linewidth=2)
plt.plot(signal_7year)
# DWW could refine by specifying hours 6pm to 6am, as years 2102 and 2013 peaks due to an inefficient old AC unit.

In [ ]:
# NEW TROUGH CODE FOR ROLLING SIGNAL DAILY OR WEEKLY: Lows less frequent w Level 1 Charger
trough_L1= df_18.kwh < 0.75
trough_signal_L1= trough_L1.resample('M').sum() # Weekly is a more stable indicator?
plt.figure(figsize= (15,10))
plt.title('Number of Hours per Week that 0.75kwh Trough L1 Signal Threshold is Crossed')
plt.ylabel('Number of Hours')
# plt.axhline(7, color='r', linestyle='dashed', linewidth=2)
plt.plot(trough_signal_L1)
# DWW NB The low values at Left and right are misleading artefacts of the resample process!!! BEWARE!!

# Another possible Signal for Classifying the new use of a Level 1 Charger is derived from total monthly power consumption

The following analysis of (resampled) monthly data explores a potential Level 1 Charger Signal.
One idea is that with the acquisition of a Level 1 charger the monthly total kwh consumption should increase.

For example, charging for 1,000 miles driven each month would add about 333 kwh per month (since 1 kwh fuels c.3 miles).

A monthly_baseline is established, which is the average of the preceding four years (so 2014-2017 here).

This is of course just one "signal" or feature. It is desireable to posit a number of such features, and then to test which combination of them is most efffective in accurately classifying a large sample of EV's.

In [ ]:
df_18.kwh.resample('M').sum().plot()

In [ ]:
df_17= dft[dft.year== 2017]
df_17.kwh.resample('M').sum().plot()

In [ ]:
df_16= dft[dft.year== 2016]
df_16.kwh.resample('M').sum().plot()

In [ ]:
df_15= dft[dft.year== 2015]
df_15.kwh.resample('M').sum().plot()

In [ ]:
df_15.kwh.resample('M').sum().values

In [ ]:
df_14= dft[dft.year== 2014]
print(df_14.kwh.resample('M').sum().values)
df_14.kwh.resample('M').sum().plot()

In [ ]:
monthly_baseline= 0.25 * (df_14.kwh.resample('M').sum().values + df_15.kwh.resample('M').sum().values + 
                   df_16.kwh.resample('M').sum().values + df_17.kwh.resample('M').sum().values)

In [ ]:
plt.plot(monthly_baseline)
plt.plot(df_18.kwh.resample('M').sum().values)

In [ ]:
plt.axhline(0, color='r', linestyle='dashed', linewidth=2) # zero reference
plt.axhline(167, color='g', linestyle='dashed', linewidth=3)# 500 miles/month minimum is an extra 167 kwh/month
plt.plot(df_18.kwh.resample('M').sum().values - monthly_baseline)

# Repeating the above analysis but just for the 12 hours when the EV is likely to be at the residence (hours 0-5, 18-23 ): this yields a sharper contrast.

In [ ]:
dft.head()
# dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']) #12 hours = half day, abbreviation H


In [ ]:
df_17H= dft[(dft.year== 2017) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_17H.head(10)

In [ ]:
df_17H.kwh.resample('M').sum().plot()

In [ ]:
df_18H= dft[(dft.year== 2018) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_18H.kwh.resample('M').sum().plot()

In [ ]:
df_18H= dft[(dft.year== 2018) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_17H= dft[(dft.year== 2017) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_16H= dft[(dft.year== 2016) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_15H= dft[(dft.year== 2015) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
df_14H= dft[(dft.year== 2014) & (dft.hour.isin(['00','01','02','03','04','05','18','19','20','21','22','23']))]
monthly_baseline_H= 0.25 * (df_14H.kwh.resample('M').sum().values + df_15H.kwh.resample('M').sum().values + 
                   df_16H.kwh.resample('M').sum().values + df_17H.kwh.resample('M').sum().values)

In [ ]:
plt.plot(monthly_baseline_H)
plt.plot(df_18H.kwh.resample('M').sum().values)

In [ ]:
# NEW CODE TO TRY  FOR MISSING CHART!
plt.axhline(0, color='r', linestyle='dashed', linewidth=2) # zero reference
plt.axhline(167, color='g', linestyle='dashed', linewidth=3)# 500 miles/month minimum is an extra 167 kwh/month
plt.plot(df_18.kwh.resample('M').sum().values - monthly_baseline_H)

# Hmmm Chart below does not look correct!! Monthly total increased by 600 kw/month !!  TOO MUCH!

In [ ]:
# BUT THIS VERSION (oddly not executed in the prior Notebook) DOES LOOK OK

plt.axhline(0, color='r', linestyle='dashed', linewidth=2)    # zero reference
plt.axhline(167, color='g', linestyle='dashed', linewidth=3)  # 500 miles/month minimum is an extra 167 kwh/month
plt.axvline(7, color='y', linestyle='dashed', linewidth=5)
plt.title('For the year 2018, Monthly Difference between Actual and the 4-Year baseline Consumption in kwh')
plt.xlabel('Month of the Year')
plt.ylabel('kwh')
plt.plot(df_18H.kwh.resample('M').sum().values - monthly_baseline_H)

# Successful Signal for Discerning the Acquisition of a Level 1 EV Charger
For the 12 hours (midnight to 6am, 6pm to midnight) that an EV is more likely to be at a residence,a monthly baseline is first established as the average of the preceding (trailing) 4 years, or 48 months.
This Baseline is then subtracted from the monthly values of the current year (for just the same 12 hours).
If the positive difference exceeds 167 kwh for three consecutive months, then the Residence is classified as owning a Level 1 EV charger. The 167 kwh/month threshold is predicated on an assumed 500 miles driven per month, and that 1 kwh fuels 3 miles of driving.

This approach can easily be generalised to the current months versus a four-year moving average.

# A Third possible Signal for Classifying the new use of a Level 1 Charger might derived from sequences of seven hours at night, or minimum threshold levels
One characteristic of a Level 1 charger (aka a "Trickle Charger) is that it is typically charging for a number of consecutive hours. For example, to provide for a 50-mile round trip would require about 17kwh, and at a charging rate of 1.9 kwh, that would take about nine consecutive hours. This might occur from 6pm to 3am, or from 9pm to 6am, for example. An approach that incorporates a seven-hour moving average might provide an indication of whether Level 1 EV charging is occuring.

# A Fourth possible Signal for Classifying the new use of a Level 1 Charger might be derived from a TOU-driven mini-spike
Where the EV charger is the customer of an electric utility that offers the option of a so-called "Time-Of-Use" (TOU) electricity tariff, EV chargers are incentivized to delay their charging until the onset of a designated "Off-Peak" period, which might start as early as 8pm or as late as midnight, and generally extends to 6am: the electricity tariff for the Off-Peak period is usually at a much lower price - even less than half the price for the preceding Peak Period. Most charging devices have timers, and the customers whohave chosen to subscribe to the TOU option therefore often set their timer to the onset of the Off-Peak period. Experience has shown that this tends to create a mini-peak. Incidentally, sometimes this option is offered to all of a utility's customers, while in other cases it may be offered exclusively to EV chargers who have signed up.

Since Corn Belt Energy does not currently offer a TOU Plan, this approach is not immediately relevant.

# An Ensemble Approach to Classification is often Superior

Four alternative signals have been outlined above that might be helpful in discerning whether a given household is an EV charger.

A statistical analysis can be performed to see which of the four Signals is superior. However,since the approaches are quite different, they may not be fully-correlated with one another. Accordingly, a mixture (ensemble approach) has often been found to be optimal.

This can be achieved in many ways, including by assigning appropriate weights to each of the different approaches.

Another way is to have a voting procedure. Suppose for example that three of the Signals are used, then each Classifier is assigned one vote, and the maority vote prevails.

# Risk Management Approach

There appears to be a significant assymettry in the cost of different outcomes (in formal terms, in the respective costs of Type I and Type II Classification errors). 

For example, suppose a household is initially classified by some judicious combination of the above-outlined Signals as having an EV charger, then the next steps may be:

(a) investigative (additional information gathering, for example), or 

(b) Promotional (making a highly attractive offer of some kind, for example to subscribe to a TOU plan, or to so-called Managed Charging).

Both of these steps are likely to be relatively inexpensive, and may represent wise precautionary investments with large associated benefits.

In contrast, consider the quite different cost of having missed one or more EV chargers served by the same local distribution transformer. Suppose that the transformer is repeatedly loaded during Peak Periods to 90% or more of its capacity, and its life is materially shortened as a result.Eventually a brown-out occurs, with two non-trivial costs. First, the transformer must be replaced, at a cost of the order of $7,500. Second, there may be repuational damage incurred, as all 5-7 households served by that transformer would have suffered from the consequences of the brown-out. Moreover, as such brown-out's become increasingly common in line with projected EV adoption levels, there could be delays in obtaining new transformers, and even in installing them, as qualified work crews become overburdened with such jobs. These costs could easily be one-hundred to one-thousand times higher.

Accordingly, it may be prudent to err on the side of caution and notionally classify a household as an EV charger even if the probability assigned by a chosen algorithm is less than 0.5. 

It would probably be sensible to accord quite different follow-up courses of action depending on whether the assigned probability was 0.3, 0.5, or 0.9. as the saying goes, "The devil is in the details".

# pane
df_18H.kwh.resample('M').sum().plot()
df_18H.kwh.resample('M').sum().plot()
df_18H.kwh.resample('M').sum().plot()
df_18H.kwh.resample('M').sum().plot()
df_18H.kwh.resample('M').sum().plot()

# A Small EV and a Level 1 "Trickle Charger" were Acquired in July, 2018
After July we see relatively little "white space" below the red horizontal line, suggesting that the trickle charger was active throughout much of the night.

In [ ]:
df_19= dft[dft.year== 2019]
plt.figure(figsize= (15,10))
df_19.kwh.plot()
plt.title('Hourly kwh Consumption: Level 2 Charger Acquired February 11th, 2019')
plt.ylabel('kwh')
plt.axhline(3.15, color='r', linestyle='dashed', linewidth=2)
plt.axhline(7, color='g', linestyle='dashed', linewidth=2)
plt.show()

# A Large (Tesla) EV and a Level 2 "Fast Charger" were Acquired in February, 2019
In mid-February and beyond we see 7 kwh of consumption frequently, while before even with the EV with the smaller battery it had been comparatively rare.

In [ ]:
sum(df_19.kwh > 7) # this includes just one month and several days of EV ownership
# NEW CODE FOR ROLLING SIGNAL DAILY OR WEEKLY
peak= df_19.kwh > 7
signal= peak.resample('W').sum() # Weekly is a more stable indicator: consistently more than 7x per week!
plt.figure(figsize= (15,10))
plt.title('Number of Times per Week that 7 kwh Signal Threshold is Exceeded')
plt.ylabel('Number of Times')
# plt.axhline(7, color='r', linestyle='dashed', linewidth=2)
plt.plot(signal)

In [ ]:
sum(df_18.kwh > 7)  # for the year, just a single occurrence

In [ ]:
sum(df_17.kwh > 7)  # for the year,not a single occurrence

In [ ]:
sum(df_16.kwh > 7)  # for the year, not a single occurrence
# Actually a number of occurences in July, 2012 when AC was old and inefficent

# Establishing a Baseline for Power Consumption using OLS Regression
The approach is to use 2017 as a baseline year for purposes of comparison.
The year 2018 is unsuitable as the numbers were materially higher in the second half of the year due to the small EV acquisition.

In [ ]:
# NEW: the following analysis takes the one-month, six day period of Tesla ownership and compares it to
# a baseline of the the same period for the preceding four years
# Can now be updated through mid July from Jeff Hevrin hopefully (uodate from Corn Belt metering data)

In [ ]:
k_19 = dft['2019-02-12':'2019-03-18']

In [ ]:
plt.plot(k_19.kwh) 

In [ ]:
plt.plot(k_19.kwh.resample('D').mean())

In [ ]:
k_19 = dft['2019-02-12':'2019-03-18'].kwh.reset_index()

In [ ]:
k_18 = dft['2018-02-12':'2018-03-18'].kwh.reset_index()
k_18[0:10]

In [ ]:
k_17 = dft['2017-02-12':'2017-03-18'].kwh.reset_index()


In [ ]:
k_16 = dft['2016-02-12':'2016-03-18'].kwh.reset_index()

In [ ]:
two_years= pd.merge(k_16, k_17, left_index= True, right_index= True)

In [ ]:
two_years.head()
two_years.columns= ['date_16','kwh_16','date_17','kwh_17']
two_years.head()

In [ ]:
three_years= pd.merge(two_years,k_18, left_index= True, right_index= True)

In [ ]:
three_years.head()
three_years.rename(columns= {'kwh_18':'kwh'})
three_years.head()

In [ ]:
four_years= pd.merge(three_years,k_19, left_index= True, right_index= True)
four_years.head()

In [ ]:
four_years.columns= ['date_16','kwh_16','date_17','kwh_17', 'date_18','kwh_18','date_19','kwh_19']
four_years.head()

In [ ]:
plt.plot(four_years.kwh_16)
plt.plot(four_years.kwh_19)


In [ ]:
four_years['baseline']= (four_years.kwh_16 + four_years.kwh_17 + four_years.kwh_18)/3
four_years= four_years.drop(['date_16','date_17','date_18'], axis= 1)
four_years.head()

In [ ]:
four_years['increment']= (four_years.kwh_19 - four_years.baseline)
four_years.head()

In [ ]:
plt.plot(four_years.baseline)
plt.plot(four_years.increment)
plt.show()

In [ ]:
four_years['hour']= four_years.date_19.dt.hour
four_years['weekday']= four_years.date_19.dt.weekday_name  # weekday to exlore if weekend charging during the day
# dft['weekday']= dft.date.dt.weekday_name
four_years.head(20)

In [ ]:
plt.hist(four_years.increment, bins=24, density= True)
plt.title('Histogram of Hourly Difference from Baseline')
plt.xlabel('Difference in kwh')
# NB Symmetric Distribution centered around zero is likely just random.
# We should therefore look at the most common hours for when the difference (aka 'increment') is more than 2.

In [ ]:
big_difference= four_years[four_years.increment > 2]
big_difference.head()

In [ ]:
plt.hist(big_difference.increment, bins=24, density= True)
plt.title('Histogram of Hourly Difference from Baseline when > 2')
plt.xlabel('Difference in kwh')

In [ ]:
gh= big_difference.groupby('hour')

In [ ]:
gh.mean()
print(gh.increment.mean())
# VERY INTERESTING THAT NO COUNTS FOR HOURS 2,3,4,5!!! CHARGING PRESUMABLY COMPLETED BY THEN??
# possibly bi-modal: hours 0 & 1 are highest, hours 6pm & 10pm are the next most common.
# matplotlib.pyplot.bar(x, height, width=0.8, bottom=None, *, align='center', data=None, **kwargs)[source]
plt.bar(gh.increment.mean(), height= 10)
#plt.plot(gh.increment.mean())
# DWW look at .sum() to give total additional 
# FAILED BAR PLOT ATTEMPT!!!


In [ ]:
gw= big_difference.groupby('weekday')
print(gw.increment.mean())
plt.plot(gw.increment.mean())  # make days properly continuous by making 'weekday' an ordered categorical variable !?
# The observed differences could just be random: no evidence of high Weekend charging: Sunday is the lowest day.

In [ ]:
g= four_years.groupby('hour')

In [ ]:
g.baseline.mean()
plt.plot(g.baseline.mean())
plt.plot(g.increment.mean())
plt.plot( g.baseline.mean() + g.increment.mean())

plt.show()
# DWW MUCH BETTER TO DO THIS AS A STACKED BAR CHART????

In [ ]:
print(g.baseline.mean())
print(g.increment.mean())

In [ ]:
# LOGARITHMIC EXPLORATIONS !!
# introduce logarithmic version!
import numpy as np 
df_17= dft[dft.year== 2017]
log= np.log(df_17.kwh)     # take log to base Ten (with e we have negative values!!)
log[0:10]

In [ ]:
# Dividing by zero error message received.
# "The proper solution here is to add some small epsilon to the argument of log function. What worked for me was

# epsilon = 1e-5    

# def cost(X, y, theta):
    m = X.shape[0]
    yp = expit(X @ theta)
    cost = - np.average(y * np.log(yp + epsilon) + (1 - y) * np.log(1 - yp + epsilon))
    return cost

In [ ]:
epsilon = 1e-5  
df_17['ln_kwh']= np.log(df_17.kwh + epsilon)
df_17.info()
df_17.head()

In [ ]:
# TEMPORAL MODEL ONLY: 2017
# This version updated by dropping the 'year' variable as redundant (as for 2017 only)
import statsmodels.api as sm
from statsmodels.formula.api import ols
m_ln= ols(formula= 'ln_kwh ~ hour + weekday + month', data= df_17).fit()  # .fit() already done (chained)




In [ ]:
print(m_ln.summary())  # Hmmm...R2 has droped from 18% to just 6.5% !!??
# this seems to be a much worse model and was examined only because of possible negative values with a non-log model
# But to simulate Accelerated Aging we are only interested in the other end of the distribution, 
# namely the very high values.
# CONCLUSION: Better to stay with the non-log formulation for now?(although variance would increase for higher values?)

# OLS Regression analysis for 2017


In [ ]:
# TEMPORAL MODEL ONLY: 2017
# This version updated by dropping the 'year' variable as redundant (as for 2017 only)
import statsmodels.api as sm
from statsmodels.formula.api import ols
m= ols(formula= 'kwh ~ hour + weekday + month', data= df_17).fit()  # .fit() already done (chained)

print(m.summary())

In [ ]:
# In Sample Prediction
kwh_pred= m.predict(df_17)
kwh_pred[0:24]
error= kwh_pred - df_17.kwh
error[0:10]
sum_of_squares= sum(error**2)
sum_of_squares
len(kwh_pred)
variance= (sum_of_squares/len(kwh_pred))
sigma= variance **0.5
sigma # for 2017 sigma = 0.72


In [ ]:
# IN SAMPLE PREDICTION WITH ERROR TERM SIMULATED
kwh_pred_w_epsilon= kwh_pred + sigma*np.random.normal(size= len(kwh_pred))
kwh_pred_w_epsilon[0:10]
# WHOOPS...we have negative hours! eg -1.36 in hour #2: therefore estimate ln kwh ?? (now negatives precluded!!)
# Properly should do a test for heteroskedacity??

In [ ]:
# Logarithmic model!?   TEMPORAL MODEL ONLY: 2017

from statsmodels.formula.api import ols
m= ols(formula= 'kwh ~ hour + weekday + month', data= df_17).fit()  # .fit() already done (chained)

print(m.summary())

In [ ]:
from datetime import datetime
import pandas as pd
f_date= pd.date_range(start= '1/1/2020', end= '12/31/2020', freq= 'H')  # f prefix for future dates: datetime object
dfF= pd.DataFrame(f_date, columns= ['f_date']) # creates df and names the column

dfF['hour']= dfF.f_date.dt.hour
dfF['weekday']= dfF.f_date.dt.weekday_name
dfF['month']= dfF.f_date.dt.month
dfF['year']= dfF.f_date.dt.year

dfF['hour']= dfF['hour'].astype('category')
dfF['weekday']= dfF['weekday'].astype('category')
dfF['month']= dfF['month'].astype('category')
dfF['year']= dfF['year'].astype('category')
dfF.head()

In [ ]:
# kwh_pred= m.predict(dfF) # data= dfF  need to add an error term, normally distributed
# + sig*random.normal(size= len(f_date))  # where sig is standard deviation of error term
import numpy as np 
epsilon= np.random.normal(size= len(f_date)) # need to import !!
epsilon[0:6]

In [ ]:
df_16= dft[dft.year== 2016]

In [ ]:
#  TEMPORAL MODEL ONLY: 2016
# Again, the redundant 'year' variable is dropped.
import statsmodels.api as sm
from statsmodels.formula.api import ols
m2= ols(formula= 'kwh ~ hour + weekday + month', data= df_16).fit()  # .fit() already done (chained)

print(m2.summary())

In [ ]:
import pandas as pd

In [ ]:
# Adding the Temperature variable
temps= pd.read_csv('chicago_hourly_temperatures.csv', usecols= [0,17])
temps.info()     # Note that the 'datetime variable is in fact just a string, and NOT a datetime object!!!
temps.head(3)

In [ ]:
import datetime as dt
temps['date']= pd.to_datetime(temps.datetime)
# raw_data['Mycol'] =  pd.to_datetime(raw_data['Mycol'], format='%d%b%Y:%H:%M:%S.%f')

In [ ]:
temps.info() # so the conversion was correctly parsed, but the hours appear to be Universal, namely Greenwich (+6)
# this is a problem as it does not match the hour in Chicago for the kwh usage !!!

In [ ]:
temps['C_date'] = temps.date - dt.timedelta(hours=6) # where 'C_date' is Chicago time.
temps.head(12)

In [ ]:
temps['hour']= temps.C_date.dt.hour
temps['month']= temps.C_date.dt.month
temps['year']= temps.C_date.dt.year
temps['F']= 1.8*(temps['Chicago'] - 273.15) + 32
temps.set_index(temps['C_date'], inplace= True)
temps.info()


In [ ]:
temps.head(12)

In [ ]:
temps= temps.drop(['datetime','Chicago','date'], axis=1)
temps['F_sqr']= temps.F**2
temps['F_cbd']= temps.F**3
temps['F_frth']= temps.F**4
temps.head(3)

In [ ]:
temps_17= temps[temps.year== 2017]

In [ ]:
temps_17.head(3)

In [ ]:
comb_17 = pd.merge(df_17, temps_17, left_index= True, right_index= True) # 'left_on= date, right_on= date)

In [ ]:
comb_17.head(3)

In [ ]:
comb_17.hour_y = comb_17.hour_y.astype('category', ordered= True)
comb_17.month_x= comb_17.month_x.astype('category', ordered= True)
comb_17.head(24)

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
m_F= ols(formula= 'kwh ~ hour_y + weekday + month_x + F + F_sqr + F_cbd + F_frth', data= comb_17).fit() 
# .fit() already done (chained)

print(m_F.summary())

# Hmmm 

In [ ]:
2**3

In [ ]:
comb_17.info()

In [ ]:
# Repeat after dropping month variable, seeing if temperature variable F is sufficient on its own
m_F_wo_month= ols(formula= 'kwh ~ hour_y + weekday + F + F_sqr + F_cbd + F_frth', data= comb_17).fit() 
# .fit() already done (chained)

print(m_F_wo_month.summary())

In [ ]:
g= temps.groupby('hour')

In [ ]:
F= 1.8*(g.mean()- 273.15) + 32
print(round(F, 1))

In [ ]:
pd.read_csv('chicago_hourly_temperatures.csv', usecols= [0,17])


In [ ]:
temps.head()

In [ ]:
comb= pd.merge()

# OLS.predict(X)
from datetime import datetime
dft['weekday']= dft.date.dt.weekday_name
dft['month']= dft.date.dt.month
dft['year']= dft.date.dt.year
dft.set_index(dft['date'], inplace= True)
dft= dft.drop(['day','date'], axis=1)
dft['hour']= dft['hour'].astype('category')
dft['weekday']= dft['weekday'].astype('category')
dft['month']= dft['month'].astype('category')
dft['year']= dft['year'].astype('category')
dft.info()
dft.head()

In [ ]:
from datetime import datetime
f_date= pd.date_range(start= '1/1/2020', end= '12/31/2020', freq= 'H')  # f prefix for future dates: datetime object
dfF= pd.DataFrame(f_dates, columns= ['f_date']) # creates df and names the column
dfF['hour']= dfF.dates.dt.month
dfF['weekday']= dfF.dates.dt.weekday_name
dfF['month']= dfF.dates.dt.month
dfF['year']= dfF.dates.dt.year
# dft.set_index(dft['date'], inplace= True)
# dft= dft.drop(['day','date'], axis=1)
dfF['hour']= dfF['hour'].astype('category')
dfF['weekday']= dfF['weekday'].astype('category')
dfF['month']= dfF['month'].astype('category')
dfF['year']= dfF['year'].astype('category')
dfF.info()
dfF.head()

In [ ]:
from datetime import datetime
dft['weekday']= dft.date.dt.weekday_name
dft['month']= dft.date.dt.month
dft['year']= dft.date.dt.year
dft.set_index(dft['date'], inplace= True)
dft= dft.drop(['day','date'], axis=1)
dft['hour']= dft['hour'].astype('category')
dft['weekday']= dft['weekday'].astype('category')
dft['month']= dft['month'].astype('category')
dft['year']= dft['year'].astype('category')
dft.info()
dft.head()

In [ ]:
from datetime import datetime
import pandas as pd
f_date= pd.date_range(start= '1/1/2020', end= '12/31/2020', freq= 'H')  # f prefix for future dates: datetime object
dfF= pd.DataFrame(f_date, columns= ['f_date']) # creates df and names the column

dfF['hour']= dfF.f_date.dt.hour
dfF['weekday']= dfF.f_date.dt.weekday_name
dfF['month']= dfF.f_date.dt.month
dfF['year']= dfF.f_date.dt.year

dfF['hour']= dfF['hour'].astype('category')
dfF['weekday']= dfF['weekday'].astype('category')
dfF['month']= dfF['month'].astype('category')
dfF['year']= dfF['year'].astype('category')
dfF.head()


In [ ]:
dfF.info()

In [ ]:
# Hmm for the purely temporal model volatility is underestimated greatly

In [ ]:
groupby hour and month mean 

In [ ]:
40000000/(42000 * 0.6)

 # OLD CODE BELOW TO BE EXCISED